In [1]:
from pymongo import MongoClient
from datetime import date

In [2]:
client = MongoClient()
db = client.bills
bill_info = db.bill_info
predictions = db.predictions
prev_predictions = db.prev_predictions


In [3]:
prev = prev_predictions.find()

In [6]:
prev[1]

{'_id': ObjectId('5c377f011417de079ba6b36f'),
 'bill_status': 'Introduced',
 'bill_text': "Making appropriations for Federal employees working during the Government shutdown beginning on or about December 22, 2018, and for other purposes. Be it enacted by the Senate and House of Representatives of the United States of America in Congress assembled, SECTION 1. SHORT TITLE. This Act may be cited as the ``No Work Without Pay Act''. SEC. 2. PAYMENT OF SALARIES OF EXCEPTED GOVERNMENT EMPLOYEES. There are appropriated, out of any money in the Treasury not otherwise appropriated, such sums as are necessary to pay, during the period of a lapse in discretionary appropriations beginning on or about December 22, 2018, the salaries and expenses of any officer or employee of the Federal Government who is excepted from the furlough resulting from such lapse. <all>",
 'body': "[Congressional Bills 116th Congress] [From the U.S. Government Publishing Office] [H.R. 271 Introduced in House (IH)] <DOC> 1

In [7]:
leg_id = 'H R 271'
cong_id = '116'

In [8]:
docs = bill_info.find({'leg_id': leg_id, 'congress_id': cong_id})

for doc in docs:
    print(doc['leg_id'])
    print('\tBill Status in database: {}'.format(doc['bill_status']))

H R 271
	Bill Status in database: Introduced


In [9]:
print('-------------------')
new = predictions.find({'leg_id': leg_id, 'congress_id': cong_id})
for n in new:
    print('{} Current Predictions'.format(n['leg_id']))
    print('\tDate: {}'.format(n['pred_date']))
    print('\t\tNLP Prediction:\t\t{}'.format(n['nlp_pred_proba']))
    print('\t\tNumerical Prediction:\t{}'.format(n['num_pred_proba']))
    print('\t\t\tCombined Prediction:\t{}'.format(n['pred_proba']))

-------------------
H R 271 Current Predictions
	Date: 01/11/2019
		NLP Prediction:		0.05664
		Numerical Prediction:	0.05
			Combined Prediction:	0.05332


In [13]:
print('-------------------')
prev = prev_predictions.find({'leg_id': leg_id, 'congress_id': cong_id})

# keep track of leg_id and it's corresponding list of tracked probabilities
leg_id_list = []
prob_track_list = []

for n in prev:
    leg_id_list.append(n['leg_id'])
    
    print('{} Previous Predictions'.format(n['leg_id']))
    if 'pred_date' in n.keys():
        pred_date = n['pred_date']
        print('\tDate: {}'.format(pred_date))
    else: 
        pred_date = '01/08/2019'
        print('\tDate: 01/08/2019')
        
    if 'nlp_pred_proba' in n.keys():
        nlp_pp = n['nlp_pred_proba']
        print('\t\tNLP Prediction:\t\t{}'.format(nlp_pp))
    else:
        nlp_pp = None
        
    if 'num_pred_proba' in n.keys():
        num_pp = n['num_pred_proba']
        print('\t\tNumerical Prediction:\t{}'.format(num_pp))
    else:
        num_pp = None
        
    if 'pred_proba' in n.keys():
        pp = n['pred_proba']
        print('\t\t\tCombined Prediction:\t{}'.format(pp))
    else:
        print('\t\t\tOld Prediction:\t\t{}'.format(n['probability']))
        
    prob_track_list.append([pred_date, nlp_pp, num_pp, pp])


-------------------
H R 271 Previous Predictions
	Date: 01/10/2019
		NLP Prediction:		0.05664
		Numerical Prediction:	0.05
			Combined Prediction:	0.05332
H R 271 Previous Predictions
	Date: 01/08/2019
			Old Prediction:		0.05664


In [14]:
leg_id_list

['H R 271', 'H R 271']

In [15]:
prob_track_list

[['01/10/2019', 0.05664, 0.05, 0.05332], ['01/08/2019', None, None, 0.05332]]

In [17]:
prob_track_list[0]

['01/10/2019', 0.05664, 0.05, 0.05332]

In [19]:
date_range = []
nlp_probs = []
num_probs = []
combined_probs = []

for p in prob_track_list:
    date_range.append(p[0])
    nlp_probs.append(p[1])
    num_probs.append(p[2])
    combined_probs.append(p[3])
    